In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

#### Fetch the data

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

website_url = requests.get(url).text
soup = BeautifulSoup(website_url, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
rows = table.text.split('\n\n\n')

data = list()
for r in rows:
    items = r.replace('\n\n', '').split('\n')
    data.append(items)

df = pd.DataFrame(data=data[1:], columns=data[0])

In [ ]:
df.head()

#### Clean the DataFrame

In [ ]:
# Rename Postcode by PostalCode
df['PostalCode'] = df['Postcode']
del df['Postcode']

# Remove Borough = Not assigned
df = df[df['Borough'] != 'Not assigned']

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
mask = (df['Neighborhood'] == 'Not assigned')
df['Neighborhood'][mask] = df['Borough']

# Combined into one row with the neighborhoods separated with a comma
transformed_df = df.groupby(['PostalCode'])['Neighborhood'].apply(','.join).reset_index()
df = pd.merge(left=transformed_df, right=df[['PostalCode', 'Borough']], on='PostalCode', how='left')
df = df[['PostalCode', 'Borough', 'Neighborhood']]
df.drop_duplicates(inplace=True)

In [ ]:
df.head()

#### Print the shape

In [ ]:
print(df.shape)

#### Get the latitude & longitude

In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

i = 0

# loop until you get the coordinates
while(lat_lng_coords is None):
    i+=1
    print(i)
    g = geocoder.google('{}, Toronto, Ontario'.format('M1H'))
    lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]